In [2]:
import sys
sys.path.append('/workspace/Documents')
import os
import nibabel as nb
import numpy as np
import pandas as pd
import shutil
import Diffusion_motion_field.functions_collection as ff

### Pick the valid ones in Pull 3
in pull 3, there are 183 cases and 16 empty folder (exclude). However, according to Dufan, only some of them are valid/picked for Q. we saved these cases into a txt file

In [15]:
txt_file = '/mnt/camca_dufan/research_data/cardiac_ct_4d/Pull_3/picked_cases.txt'
valid_list = []
with open(txt_file, 'r') as f:
    for line in f:
        line_str = line.strip()  # removes \n and spaces
        # print(line_str)  # or use it however you want
        valid_list.append(line_str)
import shutil
# copy these cases into a new folder
for a in valid_list:
    current_folder = os.path.join('/mnt/camca_dufan/research_data/cardiac_ct_4d/Pull_3/Pull_3_anon',a)
    des_folder = os.path.join('/mnt/camca_dufan/research_data/cardiac_ct_4d/Pull_3/Pull_3_picked',a)
    ff.make_folder([des_folder])
    # if os.path.isdir(des_folder) == 0:
    #     shutil.copytree(current_folder, des_folder)


In [15]:
## Put the rest in Pull 3 into Pull 4
txt_file = '/mnt/camca_dufan/research_data/cardiac_ct_4d/Pull_3/picked_cases.txt'
valid_list = []
with open(txt_file, 'r') as f:
    for line in f:
        line_str = line.strip()  # removes \n and spaces
        # print(line_str)  # or use it however you want
        valid_list.append(line_str)
# copy the cases not belong to valid list into Pull4
folders = ff.find_all_target_files(['*'],os.path.join('/mnt/camca_dufan/research_data/cardiac_ct_4d/Pull_3/Pull_3_anon'))
count = 0
for i in range(0,folders.shape[0]):
    folder = folders[i]
    base_name = os.path.basename(folder)
    if 'miss' in folder:
        continue
    if base_name in valid_list:
        continue
    des_folder = os.path.join('/mnt/camca_dufan/research_data/cardiac_ct_4d/Pull_4',base_name)
    assert isinstance(des_folder, str)
    assert isinstance(folder, str)
    files = ff.find_all_target_files(['*'],folder)  
    if os.path.isdir(des_folder) == 0:
        ff.make_folder([des_folder])
        for f in files:
            shutil.copy(f, os.path.join(des_folder, os.path.basename(f)))

    break

### Rename each case
now the case name is too long, we should re-name them into a shorter one and save into a spreadsheet. Then we rename the folders

In [4]:
main_folder = os.path.join('/mnt/camca_dufan/research_data/cardiac_ct_4d/')
pull_list = ['Pull_1', 'Pull_2', 'Pull_3','Pull_4']
count = 1
result = []
for pull in pull_list:
    if pull == 'Pull_1':
        pull_folder = os.path.join(main_folder,pull,'AnonymizedCTA_Series_Selected/studies')
    elif pull == 'Pull_2':
        pull_folder = os.path.join(main_folder,pull,'Studies_function_only')
    elif pull == 'Pull_3':
        pull_folder = os.path.join(main_folder,pull,'Pull_3_picked')  
    else:
        pull_folder = os.path.join(main_folder,pull)

    cases = ff.find_all_target_files(['*'],pull_folder)

    for case in cases:
        case_folder = case
        case_original_name = os.path.basename(case)
        case_index = 'case_'+ str(count)
        count+=1
        result.append([pull, case_index, case_original_name, case_folder])
df = pd.DataFrame(result, columns=['patient_class', 'patient_id', 'case_original_name', 'case_dicom_folder'])
file = '/mnt/camca_NAS/4DCT/Patient_lists/mgh/patient_name_correspondance.xlsx'
df.to_excel(file, index=False)

In [5]:
# rename the folder
main_path = '/mnt/camca_NAS/4DCT/mgh_data/nii-images-raw'
info = pd.read_excel('/mnt/camca_NAS/4DCT/Patient_lists/mgh/patient_name_correspondance.xlsx')
folders = ff.find_all_target_files(['Pull_4/*'],main_path)
for f in folders:
    case_id = os.path.basename(f)
    row = info[info['case_original_name'] == case_id]
    patient_class = row['patient_class'].values[0]
    patient_id = row['patient_id'].values[0]
    new_folder = os.path.join('/mnt/camca_NAS/4DCT/mgh_data/nii-images/',patient_class, patient_id)
    ff.make_folder([os.path.dirname(new_folder), new_folder])
    if os.path.isfile(os.path.join(new_folder,'img.nii.gz'))== 0:
        shutil.copy2(os.path.join(f,'img.nii.gz'), os.path.join(new_folder,'img.nii.gz'))


#### delete

In [10]:
main_path = '/mnt/camca_NAS/4DCT/mgh_data/nii-images'
folders = ff.find_all_target_files(['Pull_4/*/img.nii.gz'],main_path)
for f in folders:
    os.remove(f)